# MOrgAna workflow for advance python users
This workflow is intended for users with programming background to analyse multiple image folders at once. Users can also use this notebook to select and adapt modules/functions specific to their purpose. This workflow follows the order of scripts in python_example_scripts and explains the code shown in the scripts in more detail.

## Generate Masks

### 01_create_model_folder.py
The following code chooses images from the acquired dataset to form the training dataset for the generation of the model.

In [1]:
import os, glob, shutil
from tqdm.notebook import tqdm
import numpy as np
from numpy.random import default_rng

In [2]:
# select folder containing all image folders to be analysed
# parent_folder = os.path.join('test_data','2020-09-22_conditions')
parent_folder = os.path.join('/','Users','jialelim', 'Desktop', 'example_dataset_ipynb', 'condB')
parent_folder = os.path.join('Y:',os.sep,'Jia_Le_Lim','morgana_example_datasets','gastruloids','condA')

print('Image subfolders found in: ' + parent_folder)
if os.path.exists(parent_folder):
    print('Path exists! Proceed!')# check if the path exists

Image subfolders found in: Y:\Jia_Le_Lim\morgana_example_datasets\gastruloids\condA
Path exists! Proceed!


In [3]:
# select images for training dataset
start = 0 # increase value to exclude starting images in dataset
dN = 0 # every dNth image will be used for the training dataset; if dN = 0, random images are taken

# True: create one model for all folders; False: create one model for each image subfolder
combine_subfolders = True
   
# add folders that you want to ignore here
exclude_folder = ['model_']

In [4]:
def initialize_model_folder(folder, dN=30, start=0, combine=True):
    
    # create folders
    if combine:
        model_folder = os.path.join(os.path.split(folder)[0],'model_')
    else:
        model_folder = os.path.join(os.path.split(folder)[0], 'model_' + os.path.split(folder)[1])

    if not os.path.exists(model_folder):
        os.mkdir(model_folder)
        
    trainingset_folder = os.path.join(model_folder,'trainingset')
    if not os.path.exists(trainingset_folder):
        os.mkdir(trainingset_folder)

    # count images and extract trainingset file names
    flist = glob.glob(os.path.join(folder,'*.tif'))
    flist.sort()
    if dN:
        flist = flist[start::dN]
    else: 
        rng = default_rng()
        random_choice = rng.choice(len(flist), size=np.clip(len(flist)//10, 1, None), replace=False)
        flist = [flist[i] for i in random_choice]

    
    # copy images to trainingset folder
    for f in flist:
        fname = os.path.split(folder)[1] + '_' + os.path.split(f)[-1]
        newf = os.path.join(trainingset_folder,fname)
        if not os.path.exists(newf):
            shutil.copy(f,newf)

In [5]:
# # compute parent folder as absolute path
# parent_folder = os.path.abspath(parent_folder)
    
# # find out all image subfolders in parent_folder
# folder_names = next(os.walk(parent_folder))[1] 
    
# # exclude folders in exclude_folder
# folder_names = [g for g in folder_names if not g in exclude_folder ]

# for folder_name in tqdm(folder_names):
#     if not folder_name in exclude_folder:
#         folder_path = os.path.join(parent_folder, folder_name)

#         # for the parent_folder/every image subfolder, generate model folder and the trainingset
#         initialize_model_folder(folder_path, dN=dN, start=start, combine=combine_subfolders)

### 02_create_ground_truth.py
This script creates the binary masks (ground truths) for images copied into the trainingset folder.

In [6]:
from morgana.GUIs.manualmask import makeManualMask
from morgana.DatasetTools import io

In [7]:
import PyQt5.QtWidgets
import sys

In [8]:
def create_GT_mask(model_folder, app):
    
    ### check that model and trainingset exist
    if not os.path.exists(model_folder):
        print('Warning!')
        print(model_folder,':')
        print('Model folder not created! Skipping this subfolder.')
        return
        
    trainingset_folder = os.path.join(model_folder,'trainingset')
    if not os.path.exists(trainingset_folder):
        print('Warning!')
        print(model_folder,':')
        print('Trainingset images not found! Skipping this subfolder.')
        return

    ### load trainingset images and previously generated ground truth    
    flist_in = io.get_image_list(trainingset_folder, string_filter='_GT', mode_filter='exclude')
    flist_in.sort()
    flist_gt = io.get_image_list(trainingset_folder, string_filter='_GT', mode_filter='include')
    flist_gt.sort()

    ### if no trainingset images in the folder, skip this gastruloid
    if len(flist_in) == 0:
        print('\n\nWarning, no trainingset!','Selected "'+model_folder+'" but no trainingset *data* detected. Transfer some images in the "trainingset" folder.')
        return
    
    ### if there are more trainingset than ground truth, promptuse to make mask
    if len(flist_in)!=len(flist_gt):
        print('\n\nWarning, trainingset incomplete!','Selected "'+model_folder+'" but not all masks have been created.\nPlease provide manually annotated masks.')

        for f in flist_in:
            fn,ext = os.path.splitext(f)
            mask_name = fn+'_GT'+ext

            
            
            if not os.path.exists(mask_name):
                if not PyQt5.QtWidgets.QApplication.instance():
                    app = PyQt5.QtWidgets.QApplication(sys.argv)
                else:
                    app = PyQt5.QtWidgets.QApplication.instance() 
                m = makeManualMask(f,subfolder='',fn=fn+'_GT'+ext,wsize = (2000,2000))
                m.show()
                app.exec_()


In [9]:
model_folders = glob.glob(os.path.join(parent_folder,'model_*'))

### compute parent folder as absolute path
model_folders = [os.path.abspath(i) for i in model_folders]

app = PyQt5.QtWidgets.QApplication(sys.argv)

for model_folder in tqdm(model_folders):
    create_GT_mask(model_folder, app)

app.quit()

  0%|          | 0/1 [00:00<?, ?it/s]

Y:\Jia_Le_Lim\morgana_example_datasets\gastruloids\condA\model_


Warning, trainingset incomplete! Selected "Y:\Jia_Le_Lim\morgana_example_datasets\gastruloids\condA\model_" but not all masks have been created.
Please provide manually annotated masks.
Y:\Jia_Le_Lim\morgana_example_datasets\gastruloids\condA\model_\trainingset\condA_120h_03G.tif
Done with Y:\Jia_Le_Lim\morgana_example_datasets\gastruloids\condA\model_\trainingset\condA_120h_03G.tif
Y:\Jia_Le_Lim\morgana_example_datasets\gastruloids\condA\model_\trainingset\condA_120h_03H.tif
Done with Y:\Jia_Le_Lim\morgana_example_datasets\gastruloids\condA\model_\trainingset\condA_120h_03H.tif
Quitting
